#### 容器:对象的集合

* 容器是可迭代对象（iterable)，可迭代对象调用 iter() 函数，得到一个迭代器
* 迭代器（iterator）可以通过 next() 函数来得到下一个元素，从而支持遍历
* 生成器是一种特殊的迭代器
    - 是协程的一种重要实现方式；而 Python 3.5 引入 async await 语法糖后，生成器实现协程的方式就已经落后了

In [2]:
def is_iterable(param):
    try: 
        iter(param) 
        return True
    except TypeError:
        return False

params = [
    1234,
    '1234',
    [1, 2, 3, 4],
    set([1, 2, 3, 4]),
    {1:1, 2:2, 3:3, 4:4},
    (1, 2, 3, 4)
]
    
for param in params:
    print('{} is iterable? {}'.format(param, is_iterable(param)))

1234 is iterable? False
1234 is iterable? True
[1, 2, 3, 4] is iterable? True
{1, 2, 3, 4} is iterable? True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True
(1, 2, 3, 4) is iterable? True


#### 生成器(generator):调用 next() 函数的时候，才会生成下一个变量

* 调用 next() 函数的时候，才会生成下一个变量
* 初始化:用小括号括起来，(i for i in range(100000000))

In [3]:
import os
import psutil

# 显示当前 python 程序占用的内存大小
def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)
    
    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))

In [5]:
def test_iterator():
    show_memory_info('initing iterator')
    # 迭代器声明
    list_1 = [i for i in range(100000000)]
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')

def test_generator():
    show_memory_info('initing generator')
    # 生成器声明
    list_2 = (i for i in range(100000000))
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')

# %time test_iterator()
%time test_generator()

initing generator memory used: 74.3203125 MB
after generator initiated memory used: 74.33984375 MB
4999999950000000
after sum called memory used: 74.33984375 MB
CPU times: user 12.2 s, sys: 19 ms, total: 12.2 s
Wall time: 12.2 s


In [6]:
# 验证 (1 + 2 + 3 + ... + n)^2 = 1^3 + 2^3 + 3^3 + ... + n^3
def generator(k):
    i = 1
    while True:
        yield i ** k
        i += 1

gen_1 = generator(1)
gen_3 = generator(3)
print(gen_1)
print(gen_3)

def get_sum(n):
    sum_1, sum_3 = 0, 0
    for i in range(n):
        next_1 = next(gen_1)
        next_3 = next(gen_3)
        print('next_1 = {}, next_3 = {}'.format(next_1, next_3))
        sum_1 += next_1
        sum_3 += next_3
    print(sum_1 * sum_1, sum_3)

get_sum(8)

<generator object generator at 0x7f85ad24b1d0>
<generator object generator at 0x7f8587d336d0>
next_1 = 1, next_3 = 1
next_1 = 2, next_3 = 8
next_1 = 3, next_3 = 27
next_1 = 4, next_3 = 64
next_1 = 5, next_3 = 125
next_1 = 6, next_3 = 216
next_1 = 7, next_3 = 343
next_1 = 8, next_3 = 512
1296 1296


In [9]:
# 给定一个 list 和一个指定数字，求这个数字在 list 中的位置
#  返回一个 Generator 对象，需要使用 list 转换为列表后，才能用 print 输出
def index_generator(L, target):
    for i, num in enumerate(L):
        if num == target:
            yield i

print(list(index_generator([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)))

[2, 5, 9]


In [18]:
# 给定两个序列，判定第一个是不是第二个的子序列: 一个列表的元素在第二个列表中都按顺序出现，但是并不必挨在一起
def is_subsequence(a, b):
    b = iter(b)
    return all(i in b for i in a)

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

True
False


In [19]:
def is_subsequence(a, b):
    b = iter(b)
    print(b)

    gen = (i for i in a)
    print(gen)

    for i in gen:
        print(i)

    gen = ((i in b) for i in a)
    ## (i in b) 等价
# while True:
#     val = next(b)
#     if val == i:
#         yield True
        
        
        
    print(gen)

    for i in gen:
        print(i)

    return all(((i in b) for i in a))

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))

<generator object is_subsequence.<locals>.<genexpr> at 0x7f8587d33950>
1
3
5
<generator object is_subsequence.<locals>.<genexpr> at 0x7f8587d338d0>
True
True
True
False


In [22]:
# next() 函数运行的时候，保存了当前的指针
b = (i for i in range(5))
# print(list(b))
print(2 in b)
print(4 in b)
print(3 in b)

True
True
False
